Dans ce script, nous allons tester différents modèles de type convolution Nelle pour examiner leur performance. Nous allons ensuite tester s'ils sont robustes lorsqu'on applique une rotation à l'image d'entrée.

In [ ]:
import retinoto_py as fovea
args = fovea.Params(do_mask=True, n_train_stop=500_000, n_val_stop=10_000)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurent	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=True, do_fovea=False, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=1998, batch_size=64, num_workers=4, in_memory=True, model_name='resnet101', do_scratch=False, num_epochs=10, n_train_stop=0, n_val_stop=10000, lr=0.001, delta1=0.05, delta2=0.0, weight_decay=0.003, label_smoothing=0.0001, shuffle=True, verbose=True)

# transfer learning of each network on the  masked dataset

In [3]:
# %rm "cached_data/20_model_name=resnet101_dataset=bbox_retrained.lock"   # FORCING RECOMPUTE
# %rm "cached_data/20_model_name=resnet101_dataset=full_retrained.lock"   # FORCING RECOMPUTE
model_name = args.model_name
for dataset in fovea.all_datasets:
    name = f'20_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)
    print(50*'.')

Load JSON from pre-trained resnet cached_data/20_model_name=resnet101_dataset=full_retrained.json
cached_data/20_model_name=resnet101_dataset=full_retrained.pth: accuracy = 0.038
..................................................
Load JSON from pre-trained resnet cached_data/20_model_name=resnet101_dataset=bbox_retrained.json
cached_data/20_model_name=resnet101_dataset=bbox_retrained.pth: accuracy = 0.771
..................................................


## Model comparison

TODO Faire un nouveau notebook 23_

In [4]:
model_name = args.model_name
for dataset in fovea.all_datasets:
    results = fovea.pd.read_json(args.data_cache / f'20_model_name={model_name}_dataset={dataset}_retrained.json')
    print(model_name, dataset, results.tail(1)['acc_val'])

resnet101 full 49    0.039375
Name: acc_val, dtype: float64
resnet101 bbox 9    0.770264
Name: acc_val, dtype: float64


In [ ]:
results_13 = fovea.pd.read_json(args.data_cache / '13_model_comparison.json')
results_13

## Plot learning evolution

In [ ]:
fig, ax = fovea.plt.subplots()
for dataset in fovea.all_datasets:
    color = 'r'
    lw = 1

    name = f'20_model_name={model_name}_dataset={dataset}_retrained'
    model_filename, json_filename = fovea.do_learning(args, dataset, name)

    df_train = fovea.pd.read_json(json_filename, orient='records')

    df_train_roll = df_train.rolling(window=5, min_periods=1, center=False).mean()
    ax = df_train_roll.plot(x='total_image', y='acc_train', 
                        c=color, ls='dashed', lw=lw,
                        grid=True, ax=ax, label='TRAIN: ' + args.model_name)    
    ax = df_train_roll.plot(x='total_image', y='acc_val', 
                        c=color, lw=lw,
                        grid=True, ax=ax, label='VAL: ' + args.model_name)   

In [ ]:
df_train